# Before you start
1. **Don't edit this file, make a copy first:**
  * Click on File -> Save a copy in Drive

2. Also do the following:
  * Click on Runtime -> Change runtime type -> Make sure hardware accelerator is set to GPU

# An Overview Before We Begin
Here's a couple of important concepts to note down before we start:
- There is no one particular methodology that works best in all scenarios. This includes everything from model architecture, learning rate, loss function, and optimizer.
- Like any other engineering project, validation of what we have built is just as important as building it.

# Library Imports

In [ ]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models

from tqdm.notebook import tqdm

# Creating DataLoader
- Creating a Dataset. This bundles the data in a way that the model can understand.
- Creating a DataLoader(wraps an iterable around the dataset). This tells the model how to receive the images. Including batch_size, num_workers, shuffle configurations, etc.

## [Instantiating Transforms](https://https://pytorch.org/docs/stable/torchvision/transforms.html)

  - the transforms.Normalize([...]) function basically changes the data slightly according to the average RGB weights in the dataset.
  - This may seem a bit strange to you, why do we do this? Turns out, normalizing the data before training results in noticeable performance gains and reduction in training time. Since we generalize the data which makes it easier for the model to train
  - So why does normalizing data have such performance boosts in training? That's because by itself, the RGB values of the raw data have differing ranges. the blue pixel may have a range of of 0->125 while the red pixel may have a range of 120->245. This different range often causes headaches for the optimizer and it takes the gradient descent to converge much slowly as it has to cater to the differing conditions of both the red and blue pixel.
  - What batch normalization does is that it makes the RGB ranges somewhat similar, so the optimizer doesn't have such a hard time trying to cater for all the different ranges, and thus gradient descent covnerges faster.
  - More information here https://medium.com/@urvashilluniya/why-data-normalization-is-necessary-for-machine-learning-models-681b65a05029

Now, in the below block of code, come up with a set of tranforms for the training and validation datasets that you think might be suitable. Try using transforms such as rotation, flipping, normalizing, etc.

Find the documentation for the transforms here : https://pytorch.org/vision/stable/transforms.html

In [ ]:
# Define transforms for the training and validation set
training_transforms = transforms.Compose([...])           # Convert to tensor

validation_transforms = transforms.Compose([...])         #  Convert to tensor

## Download the MNIST Dataset
There are a number of ways to create a dataset, for example:
- Use an available [Torchvision datasets](https://pytorch.org/docs/stable/torchvision/datasets.html)
- Use ImageFolder to create a dataset from folders
- Write your own dataset as a subclass of torch.utils.data.Dataset
However, for the sake of this workshop, we are going to download the dataset from Tensorflow

In [ ]:
import tensorflow as tf
import numpy as np
# Load MNIST dataset
mnist = tf.keras.datasets.mnist

# Split the dataset into training and testing sets
(x_train, y_train), (x_val, y_val) = mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_val = x_train / 255.0, x_val / 255.0


In [ ]:
print(type(x_train))
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

<class 'numpy.ndarray'>
(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [ ]:
# a quick way to know what are the unique values in the label
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

## [Instantiating Dataset](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)
- Remember to define the ***\_\_getitem\_\_()*** method and the ***\_\_len\_\_()*** method

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, label, transforms):
        self.data = data
        self.label = label
        self.transforms = transforms

    def __getitem__(self, index):
        image = ... # get the image based on the index
        label = ... # get the label based on the index
        image = ... # apply the transformation to the image, but not the label
        return image, torch.Tensor(label).int() # the label has to be of type int

    def __len__(self):
        return ... # return the "length" of the data

In [ ]:
# we need to resize the label to a 2D array so that it can be converted to torch.Tensor
training_dataset = CustomDataset(x_train, y_train.reshape(60000, 1), training_transforms)
validation_dataset = CustomDataset(x_val, y_val.reshape(10000, 1), validation_transforms)

In [ ]:
# run some test for the training_dataset
image, label = training_dataset[0]
print(np.unique(image), image.shape, label)

## [Instantiating DataLoader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)
- This tells the model *how* how to receive the data; with the dataset as an input, along with batch size and shuffle as args

Now code up data loaders for the training and validation datasets as per the followig specs :

- In the training_loader, we're telling it batch_size = 32, and we want to shuffle the dataloader after each epoch.
- In the validation_loader, we also take batch_size = 32 but we DON'T want to shuffle the dataloader. This is because we want to be testing on the data in the same order to make sure the model really is improving and didn't hit a fluke ordering of the dataset.

In [ ]:
from torch.utils.data import DataLoader
training_loader = DataLoader(..., batch_size=..., shuffle=...)
validation_loader = DataLoader(..., batch_size=..., shuffle=...)



# Instantiating our Model
It's time to build our own model! Remember:
- Inherit the model from the **nn.Module** class
- Implement the ***forward()*** method

In this section, our model has some specific properties:
- The image has to be flattened first
- The model has 2 hidden [layers](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html), each has 16 nodes
- After each hidden layers, there is a ReLU [activation function](https://pytorch.org/docs/stable/nn.functional.html)
- After the last layer, there is a Softmax activation function

In [ ]:
class DigitRecognition(nn.Module):
    def __init__(self):
        # initialize a model
        super().__init__()
        self.flatten_layer = ...
        self.linear_layer1 = nn.Linear(..., ..., dtype=torch.float64)
        self.linear_layer2 = nn.Linear(..., ..., dtype=torch.float64)
        self.last_layer = nn.Linear(..., ..., dtype=torch.float64)
    def forward(self, x):

        x = self.flatten_layer(x)
        x = self.linear_layer1(x)
        x = ... # ReLU activation function
        x = self.linear_layer2(x)
        x = ... # ReLU activation function
        x = self.last_layer(x)
        x = ... # Softmax activation function
        return x


In [ ]:
model = DigitRecognition()

# The Training Function
- The fit() function in Pytorch is used to 'fit' the model and train it
<br>
<br>

Everytime we run through a 'batch' of data we need to do a few things
1. Clear the gradients from the previous loop  
2. Perform a forward pass (put the input through the model once)
3. Calculate the loss
4. Back propogate the loss
5. Update the parameter weights by taking a step with the optimiser

In [ ]:
# Function for the training

def train(model, train_loader, loss_fn, optimizer, device):
    model... # puts the model in training mode
    running_loss = 0
    with tqdm(total=len(train_loader)) as pbar:
        for i, data in enumerate(train_loader, 0): # loops through training data
            inputs, labels = data # separate inputs and labels (outputs),
            inputs, labels = inputs.squeeze(), labels.squeeze().type(torch.LongTensor)
            inputs, labels = ..., ... # puts the data on the GPU

            # forward + backward + optimize
            optimizer... # clear the gradients in model parameters
            outputs = ... # forward pass and get predictions, how do we get outputs?

            loss = ... # calculate loss, how do we get loss
            loss... # calculates gradient w.r.t to loss for all parameters in model that have requires_grad=True
            optimizer... # iterate over all parameters in the model with requires_grad=True and update their weights.

            running_loss += loss.item() # sum total loss in current epoch for print later

            pbar.update(1) #increment our progress bar

    return running_loss/len(train_loader) # returns the total training loss for the epoch

# The Validation Function
- A validation function is essential in any model training, because it helps you validate how well your model is performing on the validation dataset.

Note: the validation function validates the model performance by passing the entire validation set through the model ONCE. Also note that we cacluate the loss but don't propogate it back or update any weights!

In [ ]:
# Function for the validation pass

def validation(model, val_loader, loss_fn, device):
    model... # puts the model in validation mode
    running_loss = 0
    total = 0
    correct = 0

    with torch.no_grad(): # save memory by not saving gradients which we don't need
        with tqdm(total=len(val_loader)) as pbar:
            for inputs, labels in iter(val_loader):
                inputs, labels = inputs.squeeze(), labels.squeeze().type(torch.LongTensor)
                inputs, labels = ..., ... # put the data on the GPU

                outputs = ... # passes image to the model, and gets a ouput which is the class probability prediction
                val_loss = ... # calculates val_loss from model predictions and true labels
                running_loss += val_loss.item()

                _, predicted = torch.max(outputs, 1) # turns class probability predictions to class labels
                total += labels.size(0) # sums the number of predictions
                correct += (predicted == labels).sum().item() # sums the number of correct predictions

                pbar.update(1)

        return running_loss/len(val_loader), correct/total # return loss value, accuracy

#Things to note about our training and validation functions

## What's the difference between `model.train()` and `model.eval()`?
These two are extremely important to your training and validation loops. `model.eval()` takes away some layers that should only be used during training such as dropout and batch normalisation. It's important to always use `model.train()` when training and `model.eval()` when evaluating.

## Why do we need torch.no_grad()?
Running `with torch.no_grad()` means that we don't want gradients which is what happens during validation or testing, we don't need to update any gradients so we don't need to record them. Running this means that we optimize our code to not do things it doesn't need to.



# Setting Up Training
- When training models, it is substantially faster to train on NVIDIA GPU's, beacuse they offer a parallel computing platform called [cuda](https://developer.nvidia.com/cuda-zone) (cudnn is the API package to interface with cuda) that speeds up these computations exponentially.
- So here we check if cuda is available with cuda.is_available().
  - Following which, we send the model to the cuda device so the computation can be done on the GPU.

In [ ]:
%%capture
import torch.backends.cudnn as cudnn
torch.cuda.empty_cache()
cudnn.benchmark = True  # Optimise for hardware

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device) # send model to GPU

# Loss Function & Optimizers
- The **Loss Function** calculates how 'far' the model's class probability predictions are to the actual labels.
  - Notice how I'm saying "how far the model's class prob predictions are to the actual labels" instead of "how innacurate the model is", that's because accurate/inaccurate is the percentage of correctly or incorrectly predicted labels. This may sound the same to you but just keep this in mind, it will all make sense in due time.
  - CrossEntropyLoss is a way of calculating the loss of a model, other loss functions include Kullback Leibler Divergence Loss, Sparse Multiclass Cross-Entropy Loss, and much more.

- **The Optimizer** is a way of updating the weights of the model to minimize loss. In other words, the optimizer is the part of deep learning that helps a model 'learn'.
- In this case we're using the Adam optimizer, this is purely by random choice as no particular optimizer can be said to be superior to the other. There's an important concept in deep learning called "no free lunch", which means there isn't a particular methodology that will achieve the best outcome for all scenarios, what it comes down to is experimentation.
  - a lr of 0.001 is also chosen, this is usually a good learning rate start from with the Adam optimizer, however to get a more optimum learning rate, experimentation would need to be done. (The Pytorch documentation includes defaults for each different optimizer)


In [ ]:
# choose the suitable loss function and intialize Adam optimizer with learning_rate = 0.001

loss_fn = nn.... #what loss function would be good here?
optimizer = optim.Adam(..., ...)

# Let The Training Begin! Part 1


In [ ]:
total_epoch = 5 #feel free to play around with how many epochs you guys wanna do
for epoch in range(total_epoch): # loops through number of epochs
  train_loss = train(model, training_loader, loss_fn, optimizer, device)  # train the model for one epoch
  val_loss, accuracy = validation(model, validation_loader, loss_fn, device) # after training for one epoch, run the validation() function to see how the model is doing on the validation dataset
  print("Epoch: {}/{}, Training Loss: {}, Val Loss: {}, Val Accuracy: {}".format(epoch+1, total_epoch, train_loss, val_loss, accuracy))
  print('-' * 20)

print("Finished Training")

  0%|          | 0/1875 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

Epoch: 1/5, Training Loss: 1.5875242647723706, Val Loss: 1.5534178475643743, Val Accuracy: 0.9161
--------------------


  0%|          | 0/1875 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

Epoch: 2/5, Training Loss: 1.546369159459187, Val Loss: 1.5379025412021257, Val Accuracy: 0.9282
--------------------


  0%|          | 0/1875 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# We then save the model so we can come back later to it if need be
torch.save(model.state_dict(), 'stage-1')